In [5]:
from pymongo import MongoClient
import csv
import json

In [6]:
client = MongoClient()
db = client["fwc_db"]
cursor = db["apr"].aggregate([
    {"$match":{"district":"kabupaten"}},
    {"$group":{"_id":"$LOC", "NUM": {"$max":"$NUM"}, 
                       "data": {
                                "$push":{
                                    "NUM":{"$max":'$NUM'}, 
                                    "date":"$time", 
                                    "text_tweet":"$text_tweet", 
                                    "url_duplicate":"$url_duplicate",
                                    "ORG":"$ORG",
                                    "CON":"$CON"}
                                }
              }
    },
    {"$sort":{"NUM":-1}}
])

In [7]:
arr_documents = []
for data in cursor:
    obj_data = {}
    obj_data["LOC"] = data["_id"]
    obj_data["NUM"] = data["NUM"]
    obj_data["date"] = data["data"][0]["date"]
    arr_documents.append(obj_data)

In [8]:
json_string = json.dumps(arr_documents)

x = json.loads(json_string)

f = csv.writer(open("../csv/april_incident.csv", "wb+"))

# Write CSV Header, If you dont need that, remove this line
f.writerow(["location", "incident", "date"])

for x in x:
    f.writerow([x["LOC"], 
                x["NUM"],
                x["date"]])